Importing libraries

In [40]:
import pandas as pd
import sklearn as sc
import numpy as np
from collections import defaultdict
import xgboost as xgb

Our first type preprocessing stuff

In [41]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
test_ids = test["id"]

train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

train_y = train['target']
train_x = train.drop('target', axis = 1)

categorial = []
binary = []
continues = []

for f in train_x.columns:         
    # Defining the level
    if 'bin' in f:
        binary.append(f)
    elif 'cat' in f:
        categorial.append(f)
    elif train[f].dtype == float:
        continues.append(f)
    else:
        categorial.append(f)
        
for c in categorial:
    temp = pd.concat([pd.Series(train_y), pd.Series(train_x[c])],axis = 1)
    freqs = temp.groupby(by = c).agg(["mean"])
    dic = freqs.to_dict()[('target', 'mean')]
    dic = defaultdict(lambda: 0.0, dic)
    L = [dic[x] for x in train_x[c]]
    try:
        K = [dic[x] for x in test[c]]
    except(KeyError):
        print(dic)
        print(c)
        break
    test[c] = K
    train_x[c] = L

Our second type preprocessing stuff

In [42]:
def preproc(X_train):
    multreg = X_train['ps_reg_01'] * X_train['ps_reg_03'] * X_train['ps_reg_02']
    ps_car_reg = X_train['ps_car_13'] * X_train['ps_reg_03'] * X_train['ps_car_13']
    X_train = X_train.drop(['ps_calc_01', 'ps_calc_02', 'ps_calc_03', 'ps_calc_04', 'ps_calc_05', 'ps_calc_06',
                            'ps_calc_07', 'ps_calc_08', 'ps_calc_09', 'ps_calc_10', 'ps_calc_11', 'ps_calc_12',
                            'ps_calc_13', 'ps_calc_14', 'ps_calc_15_bin', 'ps_calc_16_bin', 'ps_calc_17_bin',
                            'ps_calc_18_bin', 'ps_calc_19_bin', 'ps_calc_20_bin', 'ps_car_10_cat', 'ps_ind_10_bin',
                            'ps_ind_13_bin', 'ps_ind_12_bin'], axis=1)
    X_train['mult'] = multreg
    X_train['ps_car'] = ps_car_reg
    X_train['ps_ind'] = X_train['ps_ind_03'] * X_train['ps_ind_15']
    return X_train

train_x1 = pd.read_csv('train.csv')
train_y1 = train_x1['target']
train_x1 = train_x1.drop(['id', 'target'], axis=1)
test_x1 = pd.read_csv('test.csv')
test_x1 = test_x1.drop(['id'], axis=1)

train_x1 = preproc(train_x1)
test_x1 = preproc(test_x1)

Creation of model

In [46]:
param = {}
param['objective'] = 'binary:logistic'
param['eta'] = 0.02
param['silent'] = True
param['max_depth'] = 5
param['subsample'] = 0.8
param['colsample_bytree'] = 0.8
param['eval_metric'] = 'auc'


Creation of xgboost-adapted datasets

In [51]:
dtrain = xgb.DMatrix(train_x.values, train_y.values)
dtest = xgb.DMatrix(test.values)

dtrain1 = xgb.DMatrix(train_x1.values, train_y1)
dtest1 = xgb.DMatrix(test_x1.values)

Learning

In [52]:
evallist  = [(dtrain,'eval'), (dtrain,'train')]
evallist1  = [(dtrain1,'eval'), (dtrain1,'train')]

print('Training the first...')
clf  = xgb.train(param, dtrain, 1000, evallist, early_stopping_rounds=100, maximize=True, verbose_eval=100)

print('Training the second...')
clf1 = xgb.train(param, dtrain1, 1000, evallist1, early_stopping_rounds=100, maximize=True, verbose_eval=100)

Training the first...
[0]	eval-auc:0.608587	train-auc:0.608587
Multiple eval metrics have been passed: 'train-auc' will be used for early stopping.

Will train until train-auc hasn't improved in 100 rounds.
[100]	eval-auc:0.636312	train-auc:0.636312
[200]	eval-auc:0.648939	train-auc:0.648939
[300]	eval-auc:0.659013	train-auc:0.659013
[400]	eval-auc:0.667754	train-auc:0.667754
[500]	eval-auc:0.675751	train-auc:0.675751
[600]	eval-auc:0.682439	train-auc:0.682439
[700]	eval-auc:0.688498	train-auc:0.688498
[800]	eval-auc:0.694206	train-auc:0.694206
[900]	eval-auc:0.699469	train-auc:0.699469
[999]	eval-auc:0.704642	train-auc:0.704642
Training the second...
[0]	eval-auc:0.606893	train-auc:0.606893
Multiple eval metrics have been passed: 'train-auc' will be used for early stopping.

Will train until train-auc hasn't improved in 100 rounds.
[100]	eval-auc:0.632807	train-auc:0.632807
[200]	eval-auc:0.644641	train-auc:0.644641
[300]	eval-auc:0.656568	train-auc:0.656568
[400]	eval-auc:0.665429	tr

Making prediction and saving the result

In [53]:
pred = clf.predict(dtest)
pred1 = clf1.predict(dtest1)

res = 0.5 * (pred + pred1)
res = pd.concat([test_ids, pd.Series(res)],axis = 1)
res.columns = ['id', 'target']
res.to_csv('PB1XGB1+XGB2.csv',index = False)